In [13]:
import os
import subprocess
from datetime import datetime

git_path='C:\\Users\\geesi\\Desktop\\BQ\\mall-root'

In [14]:
os.chdir(git_path)
result = subprocess.run(['git','log', 'master', 
                            '--pretty=format:"%H,%cn,%ce,%ct"', 
                            '--numstat', '--no-merges'], stdout=subprocess.PIPE)

In [15]:
raw_output = result.stdout
raw_output = raw_output.decode('utf-8')
all_lines = raw_output.split('\n')

In [16]:
def insert_commit(dbcon, hash_value, name_value, email_value, date_value, insert_value, delete_value, files_value):
    with dbcon.cursor() as cursor:
        sql = "INSERT INTO commit (`hash`, `user`, `email`, `commit_date`, `insert`, `delete`, `files`)\
        VALUES (%s, %s, %s, %s, %s, %s, %s)"
        cursor.execute(sql, (hash_value, name_value, email_value, date_value, insert_value, delete_value, files_value))
        dbcon.commit()

In [18]:
len(all_lines)

43526

In [17]:
import pymysql
from tqdm import tqdm

con = pymysql.connect(host="localhost", user="giter", passwd="giter=01", db="gitwork", charset="utf8")

i = 0
pbar = tqdm(total=len(all_lines))
while i < len(all_lines):
    if len(all_lines[i]) != 0:
        hash_, name_, email_, date_ = all_lines[i].strip('"').split(',')
        date_ = datetime.fromtimestamp(int(date_))
        i += 1
        
        all_insert = 0
        all_delete = 0
        all_file = 0
        while len(all_lines[i]) > 0:
            changes = all_lines[i].split('\t')
            if len(changes) == 3:
                insert_, delete_, filename_ = changes
                #print(insert_)
                all_insert += int(insert_) if insert_ != '-' else 0
                all_delete += int(delete_) if delete_ != '-' else 0
                all_file += 1
                
                i += 1
            else:
                break
        insert_commit(con, hash_, name_, email_, date_, all_insert, all_delete, all_file)
        pbar.update(i)
        
    if len(all_lines[i]) == 0:
        i += 1

127847761it [05:10, 684312.67it/s]                                                                                     

In [11]:
import pymysql

hash_ = '2349284732987432974'
name_ = u'纪晓茗'
email_ = 'simon@simon.com'
all_insert = 0
all_delete = 0
all_file = 0
date_ = '2018-1-1'

con = pymysql.connect(host="localhost", user="giter", passwd="giter=01", db="gitwork", charset="utf8")

In [12]:
insert_commit(con, hash_, name_, email_, date_, all_insert, all_delete, all_file)